# BART Triplet Augmentation
使用 Sciq dataset訓練 並加入 Concept 的 Triplet 資料 20 筆(From ChatGPT Candidate)<br>
直接使用 trainer 訓練 <br>

### GPU

In [1]:
!nvidia-smi

Tue Jul 11 15:09:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:01:00.0 Off |                  N/A |
| 39%   52C    P0    67W / 280W |      0MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    Off  | 00000000:02:00.0 Off |                  N/A |
| 35%   

### Weight and Bias (Assisting Metrics, Optional)

In [2]:
!pip install wandb
!wandb login
project_name = "test on Sciq with BART-Triplet-Augmentation-with-Ans-Only-Triplet"
import os

os.environ["WANDB_PROJECT"] = project_name

wandb: Currently logged in as: hankystyle. Use `wandb login --relogin` to force relogin


### import

In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

2023-07-11 15:09:39.236632: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 15:09:39.341376: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-11 15:09:39.803229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-07-11 15:09:39.803345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or direc

### Loading the dataset

In [4]:
import json
import os, sys
import fnmatch

In [5]:
def read_data(item):
    path = '../../../../data/sciq(all)/{}(fine-tune-sentence-transformer).json'.format(item)
    with open(path) as f:
        data = json.load(f)
    return data

In [6]:
train = read_data('train')
valid = read_data('valid')
test = read_data('test')

In [7]:
len(train), len(valid), len(test)

(11679, 1000, 1000)

In [8]:
train[0]

{'sentence': 'What type of organism is commonly used in preparation of foods such as cheese and yogurt?',
 'distractors': ['protozoa', 'gymnosperms', 'viruses'],
 'answer': 'mesophilic organisms',
 'triplets': [['isa', 'yogurt', 'food', 1.5036563873291016],
  ['relatedto', 'cheese', 'yogurt', 1.5036563873291016],
  ['relatedto', 'organisms', 'organism', 1.4846434891223907],
  ['antonym', 'thermophilic', 'psychrophilic', 1.3640008568763733],
  ['isa', 'cheese', 'food', 1.3373385071754456],
  ['relatedto', 'preparation', 'food', 1.3373385071754456],
  ['relatedto', 'cheese', 'food', 1.3373385071754456],
  ['relatedto', 'foods', 'food', 1.3373385071754456],
  ['relatedto', 'food', 'foods', 1.3373385071754456],
  ['antonym', 'anaerobic', 'aerobic', 1.2871821820735931],
  ['antonym', 'aerobic', 'anaerobic', 1.2871821820735931],
  ['relatedto', 'anaerobic', 'organism', 1.2871821820735931],
  ['relatedto', 'used', 'use', 1.1459934711456299]]}

### Rerank

In [9]:
for i in range(len(train)):
    d = train[i]
    sentence = d['sentence']
    answer = d['answer']
    distractors = d['distractors']
    distractors = [x.lower() for x in distractors]
    triplets = d['triplets']

    if triplets != []:
        max_score = triplets[0][-1]
        for each_triplet in triplets:
            rel, source, target, weight = each_triplet
            if source == answer or target == answer or source in distractors or target in distractors:
                each_triplet[3] = max_score + 1.0

        d['triplets'] = sorted(triplets, key = lambda x: x[3],reverse=True)

In [10]:
for i in range(len(test)):
    d = test[i]
    sentence = d['sentence']
    answer = d['answer']
    distractors = d['distractors']
    distractors = [x.lower() for x in distractors]
    triplets = d['triplets']

    d['triplets'] = sorted(triplets, key = lambda x: x[3],reverse=True)

### Filter Ans only

In [11]:
def filter(data):
    for i in range(len(data)):
        d = data[i]
        sentence = d['sentence']
        answer = d['answer']
        distractors = d['distractors']
        distractors = [x.lower() for x in distractors]
        triplets = d['triplets']

        new_triplets = []
        for each_triplet in triplets:
            rel, source, target, weight = each_triplet
            if source == answer or target == answer:
                new_triplets.append(each_triplet)
        d['triplets'] = sorted(new_triplets, key = lambda x: x[3],reverse=True)
    return data

In [12]:
train = filter(train)
valid = filter(valid)
test = filter(test)

### Prepare data

In [13]:
import random
random.seed(0)
k = 25

In [14]:
def processData(data, k):
    
    sentences = []
    labels = []
    answers = []
    for d in data:
        sentence = d['sentence']
        distractors = d['distractors']
        triplets = d['triplets']
        answer = d['answer']
        
        triplet_set = []
        for each_triplet in triplets[:k]:
            rel, source, target, weight = each_triplet
            triplet_set.append('{} {} {}'.format(source,rel,target))

        distractors = [dis.strip() for dis in distractors]

        random.shuffle(triplet_set)
        sentences.append(sentence + '</s></s>' + answer  + '</s></s>' + ', '.join(triplet_set))
        labels.append(', '.join(distractors))
        answers.append(answer)
        
    return sentences, answers, labels

In [15]:
train_sent, train_answer, train_label = processData(train, k)
valid_sent, valid_answer, valid_label = processData(valid, k)
test_sent, test_answer, test_label = processData(test, k)

In [16]:
for idx in range(2):
    print(train_sent[idx])
    print(train_answer[idx])
    print(train_label[idx])
    print()

What type of organism is commonly used in preparation of foods such as cheese and yogurt?</s></s>mesophilic organisms</s></s>
mesophilic organisms
protozoa, gymnosperms, viruses

What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?</s></s>coriolis effect</s></s>
coriolis effect
muon effect, centrifugal effect, tropical effect



In [17]:
len(train_sent), len(train_answer), len(train_label)

(11679, 11679, 11679)

In [18]:
len(valid_sent), len(valid_answer), len(valid_label)

(1000, 1000, 1000)

In [19]:
len(test_sent), len(test_answer), len(test_label)

(1000, 1000, 1000)

### Tokenization

In [20]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [21]:
train_encodings = tokenizer(train_sent, truncation=True, padding=True)
valid_encodings = tokenizer(valid_sent, truncation=True, padding=True)
test_encodings = tokenizer(test_sent, truncation=True, padding=True)

In [22]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [23]:
len(train_encodings.input_ids)

11679

In [24]:
def add_labels(encodings, distractors):
    
    distractors_encodings = tokenizer(distractors, padding=True)
    labels = []
    for i in range(len(distractors_encodings.input_ids)):
        labels.append(distractors_encodings.input_ids[i])
    
    encodings["labels"] = labels
    return encodings

In [25]:
train_encodings = add_labels(train_encodings, train_label)
valid_encodings = add_labels(valid_encodings, valid_label)
test_encodings = add_labels(test_encodings, test_label)

In [26]:
len(train_encodings.input_ids)

11679

In [27]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [28]:
tokenizer.decode(train_encodings['input_ids'][0])

'<s>What type of organism is commonly used in preparation of foods such as cheese and yogurt?</s></s>mesophilic organisms</s></s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

### Dataset

In [29]:
import torch
class SciqDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SciqDataset(train_encodings)
valid_dataset = SciqDataset(valid_encodings)
test_dataset = SciqDataset(test_encodings)

In [30]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(11679, 1000, 1000)

### Fine-tuning

In [31]:
from transformers import BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [32]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="F1@3",
    weight_decay=0.01,
    gradient_accumulation_steps = 2,
    predict_with_generate=True,
    eval_accumulation_steps = 1,
    report_to="wandb" if os.getenv("WANDB_PROJECT") else "none"
)

In [33]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [34]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # store all sentences
    predicted = []
    true_label = []
    
    for k in range(len(decoded_labels)):
        pred = decoded_preds[k]
        label = decoded_labels[k]

        pred_list = pred.split(', ')
        label_list = label.split(', ')
        

        predicted.append(pred_list)
        true_label.append(label_list)

    # evaluation metrics
    p1 = 0
    p3 = 0
    r3 = 0
    f3 = 0
    r1 = 0
    for idx in range(len(true_label)):
        distractors = predicted[idx]
        labels = true_label[idx]

        act_set = set(labels)
        pred1_set = set(distractors[:1])
        pred3_set = set(distractors[:3])

        r_1 = len(act_set & pred1_set) / float(len(act_set))
        p_1 = len(act_set & pred1_set) / float(1)
        p_3 = len(act_set & pred3_set) / float(3)
        r_3 = len(act_set & pred3_set) / float(len(act_set))

        if p_3 == 0 and r_3 == 0:
            f1_3 = 0
        else:
            f1_3 = 2 * (p_3 * r_3 / (p_3 + r_3))

        r1+=r_1
        p1+=p_1
        p3+=p_3
        r3+=r_3
        f3+=f1_3

    avg_r1 = r1 / len(true_label)
    avg_p1 = p1 / len(true_label)
    avg_p3 = p3 / len(true_label)
    avg_r3 = r3 / len(true_label)
    avg_f3 = f3 / len(true_label)

    result = {'P@1': avg_p1,
              'R@1': avg_r1,
              'P@3': avg_p3,
              'R@3': avg_r3,
              'F1@3': avg_f3}
    
    return result

In [35]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [36]:
trainer.train()

/user_data/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11679
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 9100
  Number of trainable parameters = 139420416
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hankystyle. Use `wandb login --relogin` to force relogin


/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,P@1,R@1,P@3,R@3,F1@3
0,No log,0.864818,0.067000,0.022167,0.054667,0.054286,0.054311
1,No log,0.773164,0.083000,0.027833,0.082000,0.081786,0.081711
2,1.655900,0.760023,0.104000,0.034833,0.088333,0.088286,0.088156
3,1.655900,0.756103,0.116000,0.038833,0.103667,0.103452,0.103378
4,1.655900,0.754751,0.122000,0.040833,0.107000,0.106786,0.106711
5,0.504400,0.756626,0.130000,0.043333,0.110333,0.109952,0.109978
6,0.504400,0.755200,0.137000,0.045833,0.115333,0.115286,0.115156
7,0.504400,0.759482,0.133000,0.044333,0.117000,0.116952,0.116822
8,0.453300,0.764755,0.127000,0.042167,0.118333,0.118286,0.118156
9,0.453300,0.768254,0.134000,0.044500,0.121667,0.121286,0.121311


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos

TrainOutput(global_step=9100, training_loss=0.35897905559330195, metrics={'train_runtime': 4744.8904, 'train_samples_per_second': 123.069, 'train_steps_per_second': 1.918, 'total_flos': 6.397534506295296e+16, 'train_loss': 0.35897905559330195, 'epoch': 50.0})

In [38]:
test_predictions, test_labels, test_metrics = trainer.predict(test_dataset, no_repeat_ngram_size = 2, num_beams = 6)
print('test: ')
test_metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 32
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

test: 


{'test_loss': 1.0856950283050537,
 'test_P@1': 0.149,
 'test_R@1': 0.049777777777777865,
 'test_P@3': 0.14566666666666667,
 'test_R@3': 0.1456984126984127,
 'test_F1@3': 0.14546666666666666,
 'test_runtime': 21.7782,
 'test_samples_per_second': 45.917,
 'test_steps_per_second': 1.469}

In [45]:
test_predictions, test_labels, test_metrics = trainer.predict(test_dataset)
print('test: ')
test_metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 32
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

test: 


{'test_loss': 1.0856950283050537,
 'test_P@1': 0.162,
 'test_R@1': 0.054309523809523925,
 'test_P@3': 0.14599999999999996,
 'test_R@3': 0.1464206349206349,
 'test_F1@3': 0.14603333333333332,
 'test_runtime': 17.0508,
 'test_samples_per_second': 58.648,
 'test_steps_per_second': 1.877}

In [44]:
test_predictions, test_labels, test_metrics = trainer.predict(test_dataset, no_repeat_ngram_size = 2)
print('test: ')
test_metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 32
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

test: 


{'test_loss': 1.0856950283050537,
 'test_P@1': 0.15,
 'test_R@1': 0.050333333333333424,
 'test_P@3': 0.1419999999999999,
 'test_R@3': 0.1422539682539682,
 'test_F1@3': 0.1419666666666666,
 'test_runtime': 17.2389,
 'test_samples_per_second': 58.008,
 'test_steps_per_second': 1.856}

### Save Model

In [43]:
trainer.save_model('./saved_models/k=25-epoch=50')

Saving model checkpoint to ./saved_models/k=25-epoch=50
Configuration saved in ./saved_models/k=25-epoch=50/config.json
Configuration saved in ./saved_models/k=25-epoch=50/generation_config.json
Model weights saved in ./saved_models/k=25-epoch=50/pytorch_model.bin
tokenizer config file saved in ./saved_models/k=25-epoch=50/tokenizer_config.json
Special tokens file saved in ./saved_models/k=25-epoch=50/special_tokens_map.json


In [41]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 994
  Batch size = 32
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

{'eval_loss': 0.9014971256256104,
 'eval_P@1': 0.14285714285714285,
 'eval_R@1': 0.047619047619047686,
 'eval_P@3': 0.1435278336686787,
 'eval_R@3': 0.14314458177637251,
 'eval_F1@3': 0.1431254191817572,
 'eval_runtime': 17.9077,
 'eval_samples_per_second': 55.507,
 'eval_steps_per_second': 1.787,
 'epoch': 50.0}

### Save Distractor Data

In [47]:
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

model = BartForConditionalGeneration.from_pretrained('./results/checkpoint-9100/')

loading configuration file ./results/checkpoint-9100/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LA

In [48]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="F1@3",
    weight_decay=0.01,
    predict_with_generate=True,
    gradient_accumulation_steps = 2,
    eval_accumulation_steps = 1,
    report_to="wandb" if os.getenv("WANDB_PROJECT") else "none"
)

PyTorch: setting up devices


In [49]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [50]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # store all sentences
    predicted = []
    true_label = []
    
    for k in range(len(decoded_labels)):
        pred = decoded_preds[k]
        label = decoded_labels[k]

        pred_list = pred.split(', ')
        label_list = label.split(', ')
        

        predicted.append(pred_list)
        true_label.append(label_list)

    # evaluation metrics
    p1 = 0
    p3 = 0
    r3 = 0
    f3 = 0
    r1 = 0
    for idx in range(len(true_label)):
        distractors = predicted[idx]
        labels = true_label[idx]

        act_set = set(labels)
        pred1_set = set(distractors[:1])
        pred3_set = set(distractors[:3])

        r_1 = len(act_set & pred1_set) / float(len(act_set))
        p_1 = len(act_set & pred1_set) / float(1)
        p_3 = len(act_set & pred3_set) / float(3)
        r_3 = len(act_set & pred3_set) / float(len(act_set))

        if p_3 == 0 and r_3 == 0:
            f1_3 = 0
        else:
            f1_3 = 2 * (p_3 * r_3 / (p_3 + r_3))

        r1+=r_1
        p1+=p_1
        p3+=p_3
        r3+=r_3
        f3+=f1_3

    avg_r1 = r1 / len(true_label)
    avg_p1 = p1 / len(true_label)
    avg_p3 = p3 / len(true_label)
    avg_r3 = r3 / len(true_label)
    avg_f3 = f3 / len(true_label)

    result = {'P@1': avg_p1,
              'R@1': avg_r1,
              'P@3': avg_p3,
              'R@3': avg_r3,
              'F1@3': avg_f3}
    
    return result

In [51]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### Last Epoch Result

In [52]:
test_predictions, test_labels, test_metrics = trainer.predict(test_dataset)
test_metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 32
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

{'test_loss': 1.1689198017120361,
 'test_P@1': 0.158,
 'test_R@1': 0.05261111111111121,
 'test_P@3': 0.15,
 'test_R@3': 0.14972222222222223,
 'test_F1@3': 0.14973333333333336,
 'test_runtime': 16.9947,
 'test_samples_per_second': 58.842,
 'test_steps_per_second': 1.883}

In [54]:
test_predictions, test_labels, test_metrics = trainer.predict(test_dataset, no_repeat_ngram_size = 2)
test_metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 32
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

{'test_loss': 1.1689198017120361,
 'test_P@1': 0.154,
 'test_R@1': 0.051500000000000094,
 'test_P@3': 0.1453333333333333,
 'test_R@3': 0.14505555555555552,
 'test_F1@3': 0.14506666666666662,
 'test_runtime': 17.189,
 'test_samples_per_second': 58.177,
 'test_steps_per_second': 1.862}

In [55]:
test_predictions, test_labels, test_metrics = trainer.predict(test_dataset, no_repeat_ngram_size = 2, num_beams = 6)
test_metrics

***** Running Prediction *****
  Num examples = 1000
  Batch size = 32
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

{'test_loss': 1.1689198017120361,
 'test_P@1': 0.153,
 'test_R@1': 0.051166666666666756,
 'test_P@3': 0.1456666666666666,
 'test_R@3': 0.1453888888888888,
 'test_F1@3': 0.14539999999999992,
 'test_runtime': 22.2474,
 'test_samples_per_second': 44.949,
 'test_steps_per_second': 1.438}

In [40]:
import json
def write_json(data, path):
    
    jsonString = json.dumps(data)
    jsonFile = open(path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()

In [41]:
def save_data(data, predictions, labels, file_name):

    

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # store all sentences
    predicted = []
    true_label = []
    
    for k in range(len(decoded_labels)):
        pred = decoded_preds[k]
        label = decoded_labels[k]

        pred_list = pred.split(', ')
        label_list = label.split(', ')

        predicted.append(pred_list)
        true_label.append(label_list)
    
    # evaluation metrics
    for idx in range(len(true_label)):
        
        distractors = predicted[idx]
        labels = true_label[idx]

        act_set = set(labels)
        pred1_set = set(distractors[:1])
        pred3_set = set(distractors[:3])

        p1 = len(act_set & pred1_set) / float(1)
        p3 = len(act_set & pred3_set) / float(3)
        r3 = len(act_set & pred3_set) / float(len(act_set))

        if p3 == 0 and r3 == 0:
            f1_3 = 0
        else:
            f1_3 = 2 * (p3 * r3 / (p3 + r3))
        
        data[idx]['pred_distractors'] = distractors
        data[idx]['metric'] = {'P@1': p1, 'P@3': p3, 'R@3': r3, 'F1@3': f1_3}

        if 'triplets' in data[idx]:
            del data[idx]['triplets']
    
    write_json(data, file_name)
    print(file_name + ' is saved :)')

## Save File

In [53]:
file_name = './predictions/ans_only_triplet_k={}_epoch=50.json'.format(k)
save_data(test, test_predictions, test_labels, file_name)

./predictions/ans_only_triplet_k=25_epoch=50.json is saved :)


In [64]:
decoded_preds = tokenizer.batch_decode(test_predictions, skip_special_tokens=True)

In [54]:
same_count = 0
for i in range(len(test)):
    pred = decoded_preds[i]
    pred_list = pred.split(', ')
    

    if test[i]['answer'] in pred_list:
        same_count += 1
print('答案與 distractor 相同的筆數 = ', same_count)
print('答案與 distractor 相同的比例 = ',same_count/len(test))

答案與 distractor 相同的筆數 =  101
答案與 distractor 相同的比例 =  0.10130391173520562


In [55]:
decoded_preds = tokenizer.batch_decode(test_predictions, skip_special_tokens=True)
repeat_count = 0
for k in range(len(decoded_preds)):
    pred = decoded_preds[k]
    pred_list = pred.split(', ')
    
    duplicates = set([x for x in pred_list if pred_list.count(x) > 1])
    if duplicates:
        repeat_count += 1
print('重複 distractor 筆數 = ', repeat_count)
print('重複 distractor 的比例 = ',repeat_count/len(test))

重複 distractor 筆數 =  80
重複 distractor 的比例 =  0.0802407221664995
